In [1]:
import sys
import rootpath
sys.path.append(rootpath.detect())
%pylab
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from mpl_toolkits.mplot3d.art3d import Poly3DCollection

mpl.rc('figure', figsize=[5.0, 5.0])

Using matplotlib backend: Qt5Agg
Populating the interactive namespace from numpy and matplotlib


In [2]:
%matplotlib qt

In [3]:
def same_side(V, a, b):
    """Test whether points are on the same side of a plane.
    
    The plane is defined by the points forming the _rows_ of V and the origin.
    
    Parameters
    ----------
    V : ndarray of shape (d-1, d)
        d-1 points that together with the origin define a plane 
        in d dimensions

    a : ndarray of shape (m,d)
    b : ndarray of shape (m,d)
        a[i] and b[i] are compared, for i = 1,...,m
        a and b must be 2d arrays
        
        
    Returns
    -------
    same : Bool ndarray of shape (m,)
        same[i] is True iff a[i] and b[i] are on the same side of the plane. 
    """
    assert len(a.shape) == 2 and len(b.shape) == 2
    assert a.shape == b.shape
    m, d = a.shape
    assert V.shape == (d-1, d)
    
    # Get the normal to the plane by via the SVD because the plane 
    # passes through the origin, so its distance to the origin is zero
    # The normal to the plane is the null space of the matrix of the points
    # defining the plane, including the orgin.
    
    A = np.vstack((V, np.zeros(d)))
    U, s, Vt = np.linalg.svd(A.T)
    assert s[-1] < 1e-10
    n = np.squeeze(U[:,-1])
    
    # They are on the same side if dot product with normal has the same sign
    same = np.sign(a@n) == np.sign(b@n)
    return same

### Test same_side

Points defining the plane are in blue.  The point with which others are compared (the DM's point) is red: those on the same side are orange and those on the other side are green. Rotate the figure to see which side they're really on.

In [4]:
P = rand(2, 3)  # Two random points to define a plane

npts = 8
a = np.ones((8, 3))  # Always compare to (1, 1, 1)
b = np.random.rand(8, 3)

same = same_side(P, a, b)
print(same)
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.scatter(P[:,0], P[:,1], P[:,2])

ax.add_collection3d(Poly3DCollection(np.vstack((2*P, np.zeros(3))), alpha=0.2))

ax.scatter(b[:,0], b[:,1], b[:,2], c=['r' if s else 'g' for s in same])
ax.scatter(a[0], a[1], a[2], marker='*')
ax.axes.set_xlim3d(left=0, right=1) 
ax.axes.set_ylim3d(bottom=0, top=1) 
ax.axes.set_zlim3d(bottom=0, top=1) 

[False  True  True  True  True False False  True]


(0.0, 1.0)

/home/finley/miniconda3/envs/testsuite/lib/python3.8/site-packages/numpy/lib/function_base.py:393: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis)
/home/finley/miniconda3/envs/testsuite/lib/python3.8/site-packages/numpy/core/_methods.py:161: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [5]:
from itertools import combinations


def in_cones(P, x):
    """
    Determine which cones formed from the origin and points in P
    enclose the vector x
    
    Parameters
    ----------
    
    P : ndarray, shape (m, d)
        Array of m d-dimensional points.  
        Cones are formed from combinations of d of these and the origin.
        
    x : ndarray, shape (d,)
        The ray to be enclosed. 
        
    Returns
    -------
    
    cones : list of d-dimensional tuples.
        Each tuple is the indices of the rows of P which, 
        with the origin, define the cone enclosing x.
        cones is empty if no combination of the points encloses x.
    """
    m, d = P.shape
    assert len(x.shape) == 1 and x.shape[0] == d
    
    X = np.tile(x, (m-(d-1), 1))  # Replicate x for same_side
    
    # For all possible d-dimensional planes defined by P and the origin, find 
    # whether x is on the same side as each of the remaining points in P. 
    # Store these in a dictionary whose key is a d-tuple of the point (first
    # element)and the indices of the points defining the plane.
    side = {}
    allpoints = set(range(m))
    for plane in combinations(allpoints, d-1):
        Iplane = list(plane)
        pts = allpoints.difference(set(plane))
        Ipts = list(pts)
        S = same_side(P[Iplane], P[Ipts], X)
        
        for p, same in zip(Ipts, S):
            if same:
                side[(p, *Iplane)] = 1

    # For all possible combinations of d points that, together with the origin
    # define a cone, find the ones that contain x.
    cones = []
    for simplex in combinations(allpoints, d):
        simplex = set(simplex)
        for i in simplex:
            plane = sorted(simplex.difference((i,)))
            try:
                _ = side[(i, *plane)]
            except:
                break
        else:
            # x was on the same side of all the planes
            cones.append(tuple(simplex))
    return cones

def in_cone_combinations(combos, x):
    """
    Determine which cones formed from the origin and points in P
    enclose the vector x
    
    Parameters
    ----------
    
    combos : ndarray, shape (m, d)
        Array of m d-dimensional points.  
        Cones are formed from combinations of d of these and the origin.
        
    x : ndarray, shape (d,)
        The ray to be enclosed. 
        
    Returns
    -------
    
    cones : list of d-dimensional tuples.
        Each tuple is the indices of the rows of P which, 
        with the origin, define the cone enclosing x.
        cones is empty if no combination of the points encloses x.
    """
    n_comb, m, d = P.shape
    assert len(x.shape) == 1 and x.shape[0] == d
    
    X = np.tile(x, (m-(d-1), 1))  # Replicate x for same_side
    
    # For all possible d-dimensional planes defined by cones and the origin, find 
    # whether x is on the same side as each of the remaining points in P. 
    # Store these in a dictionary whose key is a d-tuple of the point (first
    # element)and the indices of the points defining the plane.
    side = {}
    allpoints = set(range(m))
    
    for plane in combinations(allpoints, d-1):
        Iplane = list(plane)
        pts = allpoints.difference(set(plane))
        Ipts = list(pts)
        S = same_side(P[Iplane], P[Ipts], X)
        
        for p, same in zip(Ipts, S):
            if same:
                side[(p, *Iplane)] = 1

    # For all possible combinations of d points that, together with the origin
    # define a cone, find the ones that contain x.
    cones = []
    for simplex in combinations(allpoints, d):
        simplex = set(simplex)
        for i in simplex:
            plane = sorted(simplex.difference((i,)))
            try:
                _ = side[(i, *plane)]
            except:
                break
        else:
            # x was on the same side of all the planes
            cones.append(tuple(simplex))
    return cones

### Test in_cones


M mutually non-dominating points generated as random points on the surface of a sphere.
Draw filled planes for all the planes that the DM's vector intersect.

In [6]:
from itertools import cycle

def colour_cycle():
    return cycle(['tab:blue', 'tab:orange', 'tab:green', 'tab:red',
                  'tab:purple', 'tab:brown', 'tab:pink',
                  'tab:gray', 'tab:olive', 'tab:cyan'])

M = 6
np.random.seed(42)
V = np.abs(np.random.randn(M, 3))
V /= np.linalg.norm(V, axis=1)[:,np.newaxis]

dm = np.asarray([1, 2, 1], dtype='float')

cones = in_cones(V, dm)

fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.scatter(V[:,0], V[:,1], V[:,2])
for n, v in enumerate(V):
    ax.text(*v, f'{n}')


dm /= dm.max()
ax.plot([0, dm[0]], [0, dm[1]], [0, dm[2]], 'r', lw=2)

colour = colour_cycle()
for I in cones:
    I = list(I)
    Z = V[I]
    ax.add_collection3d(Poly3DCollection(Z, alpha=0.2, color=next(colour)))
    
cones

[(0, 4, 5), (1, 4, 5), (2, 4, 5), (3, 4, 5)]

In [7]:
def ssd(x, V):
    """
    Find the sum of squared distances from the 
    ray defined by x to the points in V
    
    Parameters
    ----------
    x : ndarray, shape (d,)
        Point defining a ray from the origin
        
    V : ndarray, shape (M, d)
        Rows of V define the points
        
    Returns
    -------
    
    S : float
        The sum of squared distances from the ray to V
    """
    assert V.shape[1] == x.shape[0]
    xhat = x/np.linalg.norm(x)
    
    S = 0
    for v in V:
        lenv = np.linalg.norm(v)
        vhat = v/lenv
        cs = vhat @ xhat
        S += np.sqrt(1-cs*cs)*lenv
    return S

In [8]:
S = np.asarray([ ssd(dm, V[list(cone)]) for cone in cones])
k = np.argmin(S)
print(f'Best: {k} {cones[k]}  {S[k]:.3g}')

Best: 3 (3, 4, 5)  0.94


In [9]:
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.scatter(V[:,0], V[:,1], V[:,2])
for n, v in enumerate(V):
    ax.text(*v, f'{n}')


dm /= dm.max()
ax.plot([0, dm[0]], [0, dm[1]], [0, dm[2]], 'r', lw=2)

colour = colour_cycle()
for I in cones:
    I = list(I)
    Z = V[I]
    ax.add_collection3d(Poly3DCollection(Z, alpha=0.1, color=next(colour)))

def rotate(L):
    return L[1:] + (L[0],)

for i, j in zip( cones[k], rotate(cones[k])):
    ax.plot([V[i,0], V[j,0]], [V[i,1], V[j,1]], [V[i,2], V[j,2]], 'm', lw=2)
                

### Alternative class for incrementally adding new points

In [7]:

class InCones:
    """
    Maintain a list of cones enclosing a vector, allowing incremental 
    updates to the possible vertices
    """
    def __init__(self, dm, V):
        """
        Parameters
        ----------
        
        dm : ndarray, shape (d,)
            The decision maker's vector that is enclosed.
            
        V : ndarray, shape (M, d)
            Array of M d-dimensional initial vertices with M > d
        """
        assert len(dm.shape) == 1
        self.dm = dm/np.linalg.norm(dm)
        self.d = dm.shape[0]
        self.V = V.copy()
        
        cones = in_cones(self.V, self.dm)
        if cones:
            S = np.asarray([ ssd(dm, V[list(cone)]) for cone in cones])
            k = np.argmin(S)
            self.Ibest = list(cones[k])
        else:
            self.Ibest = []
        
        # Calculate lengths
        lenv = np.linalg.norm(V, axis=1)
        cs = V @ self.dm/lenv
        self.dist = np.sqrt(1-cs*cs)*lenv
        
        
    def inside(self, I):
        """
        Return True if the vertices self.V[I] together with the origin
        form a cone contain self.dm
        
        This uses in_cones, which is a sledge hammer to crack a nut as
        all the combinations are not needed.
        """
        cones = in_cones(self.V[I], self.dm)
        return len(cones) > 0
    
    def update(self, v):
        """
        Update the possible enclosing vertices with v
        
        Parameters
        ----------
        
        v : ndarray, shape (self.d,)
        
        Returns
        -------
        
        cone: ndarray, shape (self.d, self.d)
            An array of the d-dimensional vertices which, with the origin,
            form the cone enclosing self.dm.  Each row of cone is a vertex.
        S : float
            The sum of squared distances from the DM vector to the points 
            defining the best enclosing cone.
            
            If there is no enclosing cone then (None, None) is returned.
        """
        # The criterion for the best enclosing code is that it has 
        # the minimum sum of squared distances (SSD) to the dm vector. So if 
        # v is to form the new cone, its distance to dm must be less than 
        # at least one of the vertices forming the current best cone. If 
        # it is closer then it is possible that the reduction in SSD 
        # might be so large that combinations of vertices other than those 
        # forming the previous best cone might be better. So find all the
        # combinations of vertices include v that have an SSD smaller the 
        # current best SSD.  Then find whether any of these encloses dm.
        
        lenv = np.linalg.norm(v)
        cs = v @ self.dm/lenv
        dist = np.sqrt(1-cs*cs)*lenv
        
        self.dist = np.append(self.dist, dist)
        self.V = np.append(self.V, v[np.newaxis,:], axis=0)

        if not self.Ibest:  # No enclosing cone yet, so try all combinations
            print('No enclosing cone')
            cones = in_cones(self.V, self.dm)
            if cones:
                S = np.asarray([ ssd(dm, self.V[list(cone),:]) for cone in cones])
                k = np.argmin(S)
                self.Ibest = list(cones[k])
                return self.V[self.Ibest,:], S[k]
            else:
                return None, None
       
        D = self.dist[self.Ibest]   # Distances to dm for best cone
        best_ssd = D@D
        if dist > np.max(D):
            print("Won't improve")
            # v won't improve the criterion, so return the old one.
            return self.V[self.Ibest,:], best_ssd
                
        # Try all the combinations of d-1 vertices with v 
        M, d = self.V.shape
        for others in combinations(range(M-1), d-1):
            Iverts = list((M-1, *others))
            D = self.dist[Iverts]
            S = D@D
            if S < best_ssd and self.inside(Iverts):
                self.Ibest = Iverts.copy()
                best_ssd = S
    
        return self.V[self.Ibest,:], best_ssd
    

### Test

The following cell instantiates the InCones object, and the next one adds a single point

In [59]:
M = 4
np.random.seed(42)
V = np.abs(np.random.randn(M, 3))
# V /= np.linalg.norm(V, axis=1)[:,np.newaxis]

dm = np.asarray([1, 2, 1], dtype='float')

ic = InCones(dm, V)



#### In the following cell, add a single point and draw the result.  
Repeatedly execute to add more points.

In [81]:
v = np.abs(np.random.randn(3))
# v /= np.linalg.norm(v)


cone, S = ic.update(v)

print(f'{S = }')

fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.scatter(ic.V[:,0], ic.V[:,1], ic.V[:,2])
for n, v in enumerate(ic.V):
    ax.text(*v, f'{n}')

dm /= dm.max()
ax.plot([0, dm[0]], [0, dm[1]], [0, dm[2]], 'r', lw=2)

if cone is not None:
    try:
        c = next(colour)
    except:
        colour = colour_cycle()
        c = next(colour)
    ax.add_collection3d(Poly3DCollection(cone, alpha=0.5, color=c))
    
ax.view_init(180*math.atan2(dm[2], math.hypot(dm[0], dm[1]))/math.pi, 
            180*math.atan2(dm[1], dm[0])/math.pi)

IndexError: index 6 is out of bounds for axis 0 with size 6

In [62]:
from testsuite.utilities import Pareto_split

print(ic.V.shape)
print(Pareto_split(ic.V)[0].shape)

(6, 3)
(5, 3)


In [47]:
ic.Ibest

[]

In [72]:

class FinleyInCones:
    """
    Maintain a list of cones enclosing a vector, allowing incremental 
    updates to the possible vertices
    """
    def __init__(self, dmv, y):
        """
        Parameters
        ----------
        
        dmv : ndarray, shape (d,)
            The decision maker's vector that is enclosed.
            
        y : ndarray, shape (M, d)
            Array of M d-dimensional initial vertices with M > d
        """
        assert len(dmv.shape) == 1
        self.dmv = dmv/np.linalg.norm(dmv)
        self._n_obj = dmv.shape[0]
        self.y = y.copy()
        self.p = Pareto_split(y)[0]
        
        cones = in_cones(self.p, self.dmv)
        if cones:
            S = np.asarray([ ssd(dmv, self.p[list(cone)]) for cone in cones])
            k = np.argmin(S)
            self.Ibest = list(cones[k])
        else:
            self.Ibest = []
        
        # Calculate lengths
        lenv = np.linalg.norm(self.p, axis=1)
        cs = self.p @ self.dmv/lenv
        self.dist = np.sqrt(1-cs*cs)*lenv
        
        
    def inside(self, I):
        """
        Return True if the vertices self.y[I] together with the origin
        form a cone contain self.dmv
        
        This uses in_cones, which is a sledge hammer to crack a nut as
        all the combinations are not needed.
        """
        cones = in_cones(self.p[I], self.dmv)
        return len(cones) > 0
    
    def update(self, v):
        """
        Update the possible enclosing vertices with v
        
        Parameters
        ----------
        
        v : ndarray, shape (self.d,)
        
        Returns
        -------
        
        cone: ndarray, shape (self.d, self.d)
            An array of the d-dimensional vertices which, with the origin,
            form the cone enclosing self.dmv.  Each row of cone is a vertex.
        S : float
            The sum of squared distances from the dmv vector to the points 
            defining the best enclosing cone.
            
            If there is no enclosing cone then (None, None) is returned.
        """
        # The criterion for the best enclosing code is that it has 
        # the minimum sum of squared distances (SSD) to the dmv vector. So if 
        # v is to form the new cone, its distance to dmv must be less than 
        # at least one of the vertices forming the current best cone. If 
        # it is closer then it is possible that the reduction in SSD 
        # might be so large that combinations of vertices other than those 
        # forming the previous best cone might be better. So find all the
        # combinations of vertices include v that have an SSD smaller the 
        # current best SSD.  Then find whether any of these encloses dmv.
        
        lenv = np.linalg.norm(v)
        cs = v @ self.dmv/lenv
        dist = np.sqrt(1-cs*cs)*lenv
        
        self.dist = np.append(self.dist, dist)
        self.y = np.append(self.y, v[np.newaxis,:], axis=0)

        if not self.Ibest:  # No enclosing cone yet, so try all combinations
            print('No enclosing cone')
            cones = in_cones(self.y, self.dmv)
            if cones:
                S = np.asarray([ ssd(self.dmv, self.y[list(cone),:]) for cone in cones])
                k = np.argmin(S)
                self.Ibest = list(cones[k])
                return self.y[self.Ibest,:], S[k]
            else:
                return None, None
       
        D = self.dist[self.Ibest]   # Distances to dmv for best cone
        best_ssd = D@D
        if dist > np.max(D):
            print("Won't improve")
            # v won't improve the criterion, so return the old one.
            return self.y[self.Ibest,:], best_ssd
                
        # Try all the combinations of d-1 vertices with v 
        M, d = self.y.shape
        for others in combinations(range(M-1), d-1):
            Iverts = list((M-1, *others))
            D = self.dist[Iverts]
            S = D@D
            if S < best_ssd and self.inside(Iverts):
                self.Ibest = Iverts.copy()
                best_ssd = S
    
        return self.y[self.Ibest,:], best_ssd
    

In [77]:
M = 4
np.random.seed(42)
V = np.abs(np.random.randn(M, 3))
# V /= np.linalg.norm(V, axis=1)[:,np.newaxis]

dm = np.asarray([1, 2, 1], dtype='float')

ic = FinleyInCones(dm, V)


In [80]:
v = np.abs(np.random.randn(3))
# v /= np.linalg.norm(v)


cone, S = ic.update(v)

print(f'{S = }')

fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.scatter(ic.y[:,0], ic.y[:,1], ic.y[:,2])
ax.scatter(ic.p[:,0], ic.p[:,1], ic.p[:,2], c="C1", s=150, alpha=0.3)
for n, v in enumerate(ic.y):
    ax.text(*v, f'{n}')

dm /= dm.max()
ax.plot([0, dm[0]], [0, dm[1]], [0, dm[2]], 'r', lw=2)

if cone is not None:
    try:
        c = next(colour)
    except:
        colour = colour_cycle()
        c = next(colour)
    ax.add_collection3d(Poly3DCollection(cone, alpha=0.5, color=c))
    
ax.view_init(180*math.atan2(dm[2], math.hypot(dm[0], dm[1]))/math.pi, 
            180*math.atan2(dm[1], dm[0])/math.pi)

No enclosing cone
S = 1.5605378480427154


In [79]:
print(ic.y.shape[0])
print(ic.p.shape[0])

5
3
